In [24]:
#로또번호 예측 트랜스포머

import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
import numpy as np

# CSV 파일 불러오기
data = pd.read_csv('lotto.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-6]  # 마지막 6개 행 제외 (마지막 5개 데이터와 상금 데이터)
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터


# 데이터셋 클래스 정의
class LottoDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

# 트랜스포머 모델 정의
class LottoTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(LottoTransformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(input_size, output_size)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 4
num_heads = 4
num_epochs = 100
learning_rate = 0.001
batch_size = 32

# 데이터 로더 초기화
dataset = LottoDataset(train_inputs, train_labels)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 초기화
model = LottoTransformer(input_size, hidden_size, output_size, num_layers, num_heads)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 훈련
for epoch in range(num_epochs):
    model.train()
    for batch_inputs, batch_labels in data_loader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
test_inputs = torch.tensor(scaled_data[-3:-1]).unsqueeze(0).float()  # 2022년 12월 데이터 입력

# 모델 예측
model.eval()
with torch.no_grad():
    predicted = model(test_inputs)

# 스케일링 역변환
predicted = scaler.inverse_transform(predicted.squeeze(0).numpy())

# 예측 결과 출력
lotto_numbers_prediction = predicted[-1].astype(int)[:-5]  # 마지막 5개 원소 제외
print(f'2023년 6월 로또 번호 예측: {lotto_numbers_prediction}')

Epoch [10/100], Loss: 0.1997
Epoch [20/100], Loss: 0.1042
Epoch [30/100], Loss: 0.0634
Epoch [40/100], Loss: 0.0479
Epoch [50/100], Loss: 0.0443
Epoch [60/100], Loss: 0.0427
Epoch [70/100], Loss: 0.0418
Epoch [80/100], Loss: 0.0408
Epoch [90/100], Loss: 0.0396
Epoch [100/100], Loss: 0.0379
2023년 6월 로또 번호 예측: [ 7 12 20 26 32 39 24]


In [25]:
#코스피 주가지수 예측 트랜스포머 (기본)

# CSV 파일 불러오기
data = pd.read_csv('data100.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(input_size, output_size)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001

# 모델 초기화
model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 훈련
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_inputs)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()  # 2022년 12월 데이터 입력

# 모델 예측
model.eval()
with torch.no_grad():
    predicted = model(test_inputs)

# 스케일링 역변환
predicted = scaler.inverse_transform(predicted.squeeze().unsqueeze(0).numpy())

# 예측 결과 출력
kospis_index_prediction = predicted[0][-1]  # 2023년 1월의 코스피지수

print(f'2023년 4월의 코스피지수 예측: {kospis_index_prediction:.2f}')


Epoch [100/1000], Loss: 0.0222
Epoch [200/1000], Loss: 0.0098
Epoch [300/1000], Loss: 0.0069
Epoch [400/1000], Loss: 0.0054
Epoch [500/1000], Loss: 0.0046
Epoch [600/1000], Loss: 0.0039
Epoch [700/1000], Loss: 0.0034
Epoch [800/1000], Loss: 0.0031
Epoch [900/1000], Loss: 0.0027
Epoch [1000/1000], Loss: 0.0025
2023년 4월의 코스피지수 예측: 2218.59


In [26]:
#코스피 주가지수 예측 트랜스포머 (10회 실행 후 평균 및 표준편차)

# CSV 파일 불러오기
data = pd.read_csv('data100.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(input_size, output_size)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_runs = 10  # 실행 횟수
num_epochs = 1000 #찾는 횟수
learning_rate = 0.001

predictions = []  # 예측 결과를 저장할 리스트

for _ in range(num_runs):
    # 모델 초기화
    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

    # 손실 함수와 옵티마이저 정의
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs)
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()  # 2022년 12월 데이터 입력

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs)

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze().unsqueeze(0).numpy())

    # 예측 결과 저장
    kospis_index_prediction = predicted[0][-1]  # 2023년 1월의 코스피지수
    predictions.append(kospis_index_prediction)

# 예측 결과 평균 및 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = np.mean(predictions)
std_prediction = np.std(predictions)

print(f'평균 예측값: {mean_prediction:.2f}')
print(f'표준 편차: {std_prediction:.2f}')


Epoch [100/1000], Loss: 0.0170
Epoch [200/1000], Loss: 0.0099
Epoch [300/1000], Loss: 0.0069
Epoch [400/1000], Loss: 0.0055
Epoch [500/1000], Loss: 0.0045
Epoch [600/1000], Loss: 0.0040
Epoch [700/1000], Loss: 0.0034
Epoch [800/1000], Loss: 0.0031
Epoch [900/1000], Loss: 0.0027
Epoch [1000/1000], Loss: 0.0026
Epoch [100/1000], Loss: 0.0228
Epoch [200/1000], Loss: 0.0113
Epoch [300/1000], Loss: 0.0077
Epoch [400/1000], Loss: 0.0059
Epoch [500/1000], Loss: 0.0047
Epoch [600/1000], Loss: 0.0043
Epoch [700/1000], Loss: 0.0036
Epoch [800/1000], Loss: 0.0033
Epoch [900/1000], Loss: 0.0030
Epoch [1000/1000], Loss: 0.0027
Epoch [100/1000], Loss: 0.0190
Epoch [200/1000], Loss: 0.0093
Epoch [300/1000], Loss: 0.0067
Epoch [400/1000], Loss: 0.0051
Epoch [500/1000], Loss: 0.0044
Epoch [600/1000], Loss: 0.0038
Epoch [700/1000], Loss: 0.0034
Epoch [800/1000], Loss: 0.0031
Epoch [900/1000], Loss: 0.0028
Epoch [1000/1000], Loss: 0.0026
Epoch [100/1000], Loss: 0.0192
Epoch [200/1000], Loss: 0.0092
Epoch

In [27]:
#가중치가 반영된 코스피 주가지수 예측 트랜스포머

# CSV 파일 불러오기
data = pd.read_csv('data100.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001

# 모델 초기화
model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 가중치 설정
weights = torch.randn(hidden_size)

# 모델 훈련
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_inputs, weights)  # 가중치 전달
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()  # 2022년 12월 데이터 입력

# 모델 예측
model.eval()
with torch.no_grad():
    predicted = model(test_inputs, weights)  # 가중치 전달

# 스케일링 역변환
predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

# 예측 결과 출력
kospis_index_prediction = predicted[-1, -1]
print(f'Predicted KOSPI Index: {kospis_index_prediction:.2f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0268
Epoch [200/1000], Loss: 0.0136
Epoch [300/1000], Loss: 0.0093
Epoch [400/1000], Loss: 0.0071
Epoch [500/1000], Loss: 0.0057
Epoch [600/1000], Loss: 0.0051
Epoch [700/1000], Loss: 0.0043
Epoch [800/1000], Loss: 0.0040
Epoch [900/1000], Loss: 0.0035
Epoch [1000/1000], Loss: 0.0031
Predicted KOSPI Index: 2266.62


In [28]:
#코스피 주가지수의 증감 퍼센트를 확인하는 트랜스포머

# CSV 파일 불러오기
data = pd.read_csv('percent.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
num_runs = 10

predictions = []

for _ in range(num_runs):

    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)


    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


    weights = torch.randn(hidden_size)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs, weights)  # 가중치 전달
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()  # 2022년 12월 데이터 입력

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs, weights)  # 가중치 전달

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

    # 예측 결과 저장
    predictions.append(predicted[-1, -1])

# 예측 결과의 평균과 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = predictions.mean()
std_prediction = predictions.std()

print(f'Mean of Predicted KOSPI Index: {mean_prediction:.2f}')
print(f'Standard Deviation of Predicted KOSPI Index: {std_prediction:.2f}')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0211
Epoch [200/1000], Loss: 0.0191
Epoch [300/1000], Loss: 0.0181
Epoch [400/1000], Loss: 0.0167
Epoch [500/1000], Loss: 0.0150
Epoch [600/1000], Loss: 0.0135
Epoch [700/1000], Loss: 0.0118
Epoch [800/1000], Loss: 0.0107
Epoch [900/1000], Loss: 0.0096
Epoch [1000/1000], Loss: 0.0086


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0220
Epoch [200/1000], Loss: 0.0201
Epoch [300/1000], Loss: 0.0190
Epoch [400/1000], Loss: 0.0180
Epoch [500/1000], Loss: 0.0164
Epoch [600/1000], Loss: 0.0148
Epoch [700/1000], Loss: 0.0131
Epoch [800/1000], Loss: 0.0117
Epoch [900/1000], Loss: 0.0101
Epoch [1000/1000], Loss: 0.0092


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch [100/1000], Loss: 0.0207
Epoch [200/1000], Loss: 0.0194
Epoch [300/1000], Loss: 0.0182
Epoch [400/1000], Loss: 0.0172
Epoch [500/1000], Loss: 0.0155
Epoch [600/1000], Loss: 0.0140
Epoch [700/1000], Loss: 0.0124
Epoch [800/1000], Loss: 0.0111
Epoch [900/1000], Loss: 0.0096
Epoch [1000/1000], Loss: 0.0087


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0216
Epoch [200/1000], Loss: 0.0199
Epoch [300/1000], Loss: 0.0188
Epoch [400/1000], Loss: 0.0177
Epoch [500/1000], Loss: 0.0170
Epoch [600/1000], Loss: 0.0152
Epoch [700/1000], Loss: 0.0139
Epoch [800/1000], Loss: 0.0123
Epoch [900/1000], Loss: 0.0109
Epoch [1000/1000], Loss: 0.0098


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0214
Epoch [200/1000], Loss: 0.0198
Epoch [300/1000], Loss: 0.0188
Epoch [400/1000], Loss: 0.0177
Epoch [500/1000], Loss: 0.0170
Epoch [600/1000], Loss: 0.0156
Epoch [700/1000], Loss: 0.0145
Epoch [800/1000], Loss: 0.0130
Epoch [900/1000], Loss: 0.0119
Epoch [1000/1000], Loss: 0.0106
Mean of Predicted KOSPI Index: -0.94
Standard Deviation of Predicted KOSPI Index: 4.87


In [29]:
#코스피 주가지수의 증감 퍼센트 및 가중치와 10회 실행 후 평균과 표준편차를 구하는 트랜스포머

# CSV 파일 불러오기
data = pd.read_csv('percent.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
num_runs = 10

predictions = []

for _ in range(num_runs):

    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)


    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


    weights = torch.randn(hidden_size)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs, weights)  # 가중치 전달
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()  # 2022년 12월 데이터 입력

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs, weights)  # 가중치 전달

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

    # 예측 결과 저장
    predictions.append(predicted[-1, -1])

# 예측 결과의 평균과 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = predictions.mean()
std_prediction = predictions.std()

print(f'Mean of Predicted KOSPI Index: {mean_prediction:.2f}')
print(f'Standard Deviation of Predicted KOSPI Index: {std_prediction:.2f}')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0213
Epoch [200/1000], Loss: 0.0198
Epoch [300/1000], Loss: 0.0185
Epoch [400/1000], Loss: 0.0172
Epoch [500/1000], Loss: 0.0160
Epoch [600/1000], Loss: 0.0147
Epoch [700/1000], Loss: 0.0133
Epoch [800/1000], Loss: 0.0122
Epoch [900/1000], Loss: 0.0110
Epoch [1000/1000], Loss: 0.0100


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0203
Epoch [200/1000], Loss: 0.0192
Epoch [300/1000], Loss: 0.0185
Epoch [400/1000], Loss: 0.0170
Epoch [500/1000], Loss: 0.0152
Epoch [600/1000], Loss: 0.0137
Epoch [700/1000], Loss: 0.0124
Epoch [800/1000], Loss: 0.0110
Epoch [900/1000], Loss: 0.0099
Epoch [1000/1000], Loss: 0.0090


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch [100/1000], Loss: 0.0201
Epoch [200/1000], Loss: 0.0191
Epoch [300/1000], Loss: 0.0179
Epoch [400/1000], Loss: 0.0162
Epoch [500/1000], Loss: 0.0143
Epoch [600/1000], Loss: 0.0121
Epoch [700/1000], Loss: 0.0105
Epoch [800/1000], Loss: 0.0093
Epoch [900/1000], Loss: 0.0082
Epoch [1000/1000], Loss: 0.0074


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0219
Epoch [200/1000], Loss: 0.0201
Epoch [300/1000], Loss: 0.0186
Epoch [400/1000], Loss: 0.0182
Epoch [500/1000], Loss: 0.0171
Epoch [600/1000], Loss: 0.0158
Epoch [700/1000], Loss: 0.0145
Epoch [800/1000], Loss: 0.0132
Epoch [900/1000], Loss: 0.0118
Epoch [1000/1000], Loss: 0.0108


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0219
Epoch [200/1000], Loss: 0.0198
Epoch [300/1000], Loss: 0.0187
Epoch [400/1000], Loss: 0.0172
Epoch [500/1000], Loss: 0.0158
Epoch [600/1000], Loss: 0.0149
Epoch [700/1000], Loss: 0.0133
Epoch [800/1000], Loss: 0.0121
Epoch [900/1000], Loss: 0.0109
Epoch [1000/1000], Loss: 0.0100
Mean of Predicted KOSPI Index: -2.42
Standard Deviation of Predicted KOSPI Index: 3.32


In [30]:
#위 코드에서 loss함수를 Huber Loss로 대체한 코드

# CSV 파일 불러오기
data = pd.read_csv('percent.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
num_runs = 10

predictions = []

for _ in range(num_runs):

    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

    criterion = nn.SmoothL1Loss()  # Huber Loss로 대체

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    weights = torch.randn(hidden_size)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs, weights)  # 가중치 전달
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()  # 2022년 12월 데이터 입력

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs, weights)  # 가중치 전달

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

    # 예측 결과 저장
    predictions.append(predicted[-1, -1])

# 예측 결과의 평균과 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = predictions.mean()
std_prediction = predictions.std()

print(f'Mean of Predicted KOSPI Index: {mean_prediction:.2f}')
print(f'Standard Deviation of Predicted KOSPI Index: {std_prediction:.2f}')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch [100/1000], Loss: 0.0108
Epoch [200/1000], Loss: 0.0100
Epoch [300/1000], Loss: 0.0092
Epoch [400/1000], Loss: 0.0086
Epoch [500/1000], Loss: 0.0077
Epoch [600/1000], Loss: 0.0069
Epoch [700/1000], Loss: 0.0061
Epoch [800/1000], Loss: 0.0055
Epoch [900/1000], Loss: 0.0049
Epoch [1000/1000], Loss: 0.0044


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch [100/1000], Loss: 0.0106
Epoch [200/1000], Loss: 0.0098
Epoch [300/1000], Loss: 0.0093
Epoch [400/1000], Loss: 0.0089
Epoch [500/1000], Loss: 0.0085
Epoch [600/1000], Loss: 0.0078
Epoch [700/1000], Loss: 0.0068
Epoch [800/1000], Loss: 0.0060
Epoch [900/1000], Loss: 0.0052
Epoch [1000/1000], Loss: 0.0047


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


...

Epoch [100/1000], Loss: 0.0109
Epoch [200/1000], Loss: 0.0099
Epoch [300/1000], Loss: 0.0093
Epoch [400/1000], Loss: 0.0086
Epoch [500/1000], Loss: 0.0079
Epoch [600/1000], Loss: 0.0070
Epoch [700/1000], Loss: 0.0063
Epoch [800/1000], Loss: 0.0056
Epoch [900/1000], Loss: 0.0050
Epoch [1000/1000], Loss: 0.0045


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch [100/1000], Loss: 0.0111
Epoch [200/1000], Loss: 0.0101
Epoch [300/1000], Loss: 0.0094
Epoch [400/1000], Loss: 0.0091
Epoch [500/1000], Loss: 0.0087
Epoch [600/1000], Loss: 0.0082
Epoch [700/1000], Loss: 0.0075
Epoch [800/1000], Loss: 0.0070
Epoch [900/1000], Loss: 0.0063
Epoch [1000/1000], Loss: 0.0058


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch [100/1000], Loss: 0.0109
Epoch [200/1000], Loss: 0.0101
Epoch [300/1000], Loss: 0.0094
Epoch [400/1000], Loss: 0.0090
Epoch [500/1000], Loss: 0.0086
Epoch [600/1000], Loss: 0.0079
Epoch [700/1000], Loss: 0.0071
Epoch [800/1000], Loss: 0.0064
Epoch [900/1000], Loss: 0.0057
Epoch [1000/1000], Loss: 0.0051
Mean of Predicted KOSPI Index: -2.96
Standard Deviation of Predicted KOSPI Index: 2.59
